In [5]:
import sys; sys.path.append('..')
from osp import *
pd.options.display.max_colwidth = 200
pd.options.display.max_rows = 7

In [3]:
df_preds, df_feats, d_models = get_preds_feats(_force=True)

## 1925-1950 Philosophy vs 1925-1950 Literature


100%|██████████| 10/10 [00:15<00:00,  1.57s/it]


## 1950-1975 Philosophy vs 1950-1975 Literature


100%|██████████| 10/10 [00:12<00:00,  1.22s/it]


## 1975-2000 Philosophy vs 1975-2000 Literature


100%|██████████| 10/10 [00:13<00:00,  1.30s/it]


## 2000-2025 Philosophy vs 2000-2025 Literature


100%|██████████| 10/10 [00:13<00:00,  1.40s/it]


In [6]:
df_all_feats = get_all_feats(normalize=True)
df_all_feats_raw = get_all_feats(normalize=False)

In [7]:
df_all_feats_raw_means = df_all_feats_raw.mean()
df_all_feats_raw_stds = df_all_feats_raw.std()

In [8]:
newtext= """By a `denoting phrase' I mean a phrase such as any one of the following: a man, some man, any man, every man, all men, the present King of England, the presenting King of France, the center of mass of the solar system at the first instant of the twentieth century, the revolution of the earth round the sun, the revolution of the sun round the earth. Thus a phrase is denoting solely in virtue of its form. We may distinguish three cases: (1) A phrase may be denoting, and yet not denote anything; e.g., `the present King of France'. (2) A phrase may denote one definite object; e.g., `the present King of England' denotes a certain man. (3) A phrase may denote ambiguously; e.g. `a man' denotes not many men, but an ambiguous man. The interpretation of such phrases is a matter of considerably difficulty; indeed, it is very hard to frame any theory not susceptible of formal refutation. All the difficulties with which I am acquainted are met, so far as I can discover, by the theory which I am about to explain."""

In [9]:
doc = get_nlp_doc(newtext)

In [13]:
# new_feats = extract_slice_feats(doc)

In [12]:
# new_feats

In [69]:
new_feats_z = {
    k: float((v - df_all_feats_raw_means[k]) / df_all_feats_raw_stds[k])
    for k, v in new_feats.items()
    if k in df_all_feats_raw_means
}
# new_feats_z

In [71]:
df_preds.columns

Index(['true_label', 'pred_label', 'prob1', 'prob2', 'test_label',
       'confidence', 'correct', 'accuracy', 'support', 'run', 'predict_type',
       'comparison'],
      dtype='object')

In [73]:
def get_feat_names_from_models(d_models):
    for cmpname, models in d_models.items():
        for mdl in models:
            return mdl.features_

In [77]:
ld_feats_new = []
ld_preds_new = []
feat_names = get_feat_names_from_models(d_models)
for cmpname, cdf in df_feats.groupby('comparison'):
    for feat in feat_names:
        d_feats_new = dict(df_feats.query('feature==@feat & comparison==@cmpname').iloc[0])
        d_feats_new['score_mean3'] = new_feats.get(feat, 0)
        d_feats_new['score_z3'] = new_feats_z.get(feat, 0)
        ld_feats_new.append(d_feats_new)

df_feats_new = pd.DataFrame(ld_feats_new)
df_feats_new['score_mean_diff_3-1'] = df_feats_new['score_mean3'] - df_feats_new['score_mean1']
df_feats_new['score_mean_diff_3-2'] = df_feats_new['score_mean3'] - df_feats_new['score_mean2']
df_feats_new['score_mean_diff_abs_3-1'] = df_feats_new['score_mean_diff_3-1'].abs()
df_feats_new['score_mean_diff_abs_3-2'] = df_feats_new['score_mean_diff_3-2'].abs()
df_feats_new['score_z_diff_3-1'] = df_feats_new['score_z3'] - df_feats_new['score_z1']
df_feats_new['score_z_diff_3-2'] = df_feats_new['score_z3'] - df_feats_new['score_z2']
df_feats_new['score_z_diff_abs_3-1'] = df_feats_new['score_z_diff_3-1'].abs()
df_feats_new['score_z_diff_abs_3-2'] = df_feats_new['score_z_diff_3-2'].abs()

In [79]:
df_preds.columns

Index(['true_label', 'pred_label', 'prob1', 'prob2', 'test_label',
       'confidence', 'correct', 'accuracy', 'support', 'run', 'predict_type',
       'comparison'],
      dtype='object')

In [87]:
ld_preds_new = []
for cmpname, models in d_models.items():
    for nrun, mdl in enumerate(models):
        new_prob1,new_prob2 = mdl.predict_proba(new_feats_z_df.values)[0]
        new_pred = mdl.predict(new_feats_z_df.values)[0]
        d_preds_new = {
            'comparison': cmpname,
            'run':nrun,
            'predict_type': 'new',
            'test_label': ' / '.join(mdl.classes_),
            'true_label': '',
            'pred_label': new_pred,
            'prob1': new_prob1,
            'prob2': new_prob2,
        }
        ld_preds_new.append(d_preds_new)
df_preds_new = pd.DataFrame(ld_preds_new)
df_preds_new

,comparison,run,predict_type,test_label,true_label,pred_label,prob1,prob2
0,1925-1950 Philosophy vs 1925-1950 Literature,0,new,Literature / Philosophy,,Literature,8.716178e-01,0.128382
1,1925-1950 Philosophy vs 1925-1950 Literature,1,new,Literature / Philosophy,,Philosophy,4.674318e-01,0.532568
2,1925-1950 Philosophy vs 1925-1950 Literature,2,new,Literature / Philosophy,,Literature,8.776232e-01,0.122377
...,...,...,...,...,...,...,...,...
37,2000-2025 Philosophy vs 2000-2025 Literature,7,new,Literature / Philosophy,,Philosophy,3.519065e-07,1.000000
38,2000-2025 Philosophy vs 2000-2025 Literature,8,new,Literature / Philosophy,,Philosophy,6.490043e-07,0.999999
39,2000-2025 Philosophy vs 2000-2025 Literature,9,new,Literature / Philosophy,,Philosophy,6.568179e-04,0.999343


In [95]:
df_preds_new.groupby(['comparison','pred_label']).size()

comparison                                    pred_label
1925-1950 Philosophy vs 1925-1950 Literature  Literature     8
                                              Philosophy     2
1950-1975 Philosophy vs 1950-1975 Literature  Philosophy    10
1975-2000 Philosophy vs 1975-2000 Literature  Philosophy    10
2000-2025 Philosophy vs 2000-2025 Literature  Philosophy    10
dtype: int64

In [96]:
df_feats_new = df_feats_new.sort_values('score_z_diff_3-1', ascending=False)
df_feats_new[['feature', 'score_mean1', 'score_mean2', 'score_mean3', 'score_z1', 'score_z2', 'score_z3', 'score_mean_diff_3-1', 'score_z_diff_3-1', 'score_mean_diff_abs_3-1', 'score_z_diff_abs_3-1']]

,feature,score_mean1,score_mean2,score_mean3,score_z1,score_z2,score_z3,score_mean_diff_3-1,score_z_diff_3-1,score_mean_diff_abs_3-1,score_z_diff_abs_3-1
33,pos_FW,0.681976,0.450712,13.157895,0.057942,-0.170174,10.762948,12.475919,10.705007,12.475919,10.705007
227,pos_FW,0.811822,0.172755,13.157895,0.160788,-0.409325,10.762948,12.346073,10.602160,12.346073,10.602160
130,pos_FW,0.898988,0.257003,13.157895,0.235323,-0.336839,10.762948,12.258907,10.527625,12.258907,10.527625
...,...,...,...,...,...,...,...,...,...,...,...
239,deprel_obl,44.843042,48.306430,17.543860,-0.200810,0.220577,-3.582871,-27.299183,-3.382060,27.299183,3.382060
142,deprel_obl,45.134007,49.125822,17.543860,-0.156063,0.321886,-3.582871,-27.590147,-3.426808,27.590147,3.426808
45,deprel_obl,47.548407,49.383174,17.543860,0.109448,0.353705,-3.582871,-30.004548,-3.692319,30.004548,3.692319


In [97]:
df_preds

,true_label,pred_label,prob1,prob2,test_label,confidence,correct,accuracy,support,run,predict_type,comparison
id,,,,,,,,,,,,
lit/459228__02,Literature,Literature,0.988514,0.011486,Literature / Philosophy,0.988514,1,0.9135,2000.0,0,cv,1925-1950 Philosophy vs 1925-1950 Literature
lit/433975__01,Literature,Literature,0.999985,0.000015,Literature / Philosophy,0.999985,1,0.9135,2000.0,0,cv,1925-1950 Philosophy vs 1925-1950 Literature
lit/508528__02,Literature,Literature,0.999945,0.000055,Literature / Philosophy,0.999945,1,0.9135,2000.0,0,cv,1925-1950 Philosophy vs 1925-1950 Literature
...,...,...,...,...,...,...,...,...,...,...,...,...
phil/10.2307/2953730__03,Philosophy,Philosophy,0.193702,0.806298,Literature / Philosophy,NaN,1,NaN,NaN,9,unseen,2000-2025 Philosophy vs 2000-2025 Literature
lit/468421__06,Literature,Philosophy,0.227231,0.772769,Literature / Philosophy,NaN,0,NaN,NaN,9,unseen,2000-2025 Philosophy vs 2000-2025 Literature
phil/10.2307/2185617__06,Philosophy,Philosophy,0.481496,0.518504,Literature / Philosophy,NaN,1,NaN,NaN,9,unseen,2000-2025 Philosophy vs 2000-2025 Literature


In [98]:
def get_new_preds_feats(txt):
    doc = get_nlp_doc(txt)

    # get feats
    new_feats = extract_slice_feats(doc)
    new_feats_z = {
        k: float((v - df_all_feats_raw_means[k]) / df_all_feats_raw_stds[k])
        for k, v in new_feats.items()
        if k in df_all_feats_raw_means
    }

    # reintegrate feats into df_feats
    ld_feats_new = []
    feat_names = get_feat_names_from_models(d_models)
    for cmpname, cdf in df_feats.groupby('comparison'):
        for feat in feat_names:
            d_feats_new = dict(df_feats.query('feature==@feat & comparison==@cmpname').iloc[0])
            d_feats_new['score_mean3'] = new_feats.get(feat, 0)
            d_feats_new['score_z3'] = new_feats_z.get(feat, 0)
            ld_feats_new.append(d_feats_new)

    df_feats_new = pd.DataFrame(ld_feats_new)
    df_feats_new['score_mean_diff_3-1'] = df_feats_new['score_mean3'] - df_feats_new['score_mean1']
    df_feats_new['score_mean_diff_3-2'] = df_feats_new['score_mean3'] - df_feats_new['score_mean2']
    df_feats_new['score_mean_diff_abs_3-1'] = df_feats_new['score_mean_diff_3-1'].abs()
    df_feats_new['score_mean_diff_abs_3-2'] = df_feats_new['score_mean_diff_3-2'].abs()
    df_feats_new['score_z_diff_3-1'] = df_feats_new['score_z3'] - df_feats_new['score_z1']
    df_feats_new['score_z_diff_3-2'] = df_feats_new['score_z3'] - df_feats_new['score_z2']
    df_feats_new['score_z_diff_abs_3-1'] = df_feats_new['score_z_diff_3-1'].abs()
    df_feats_new['score_z_diff_abs_3-2'] = df_feats_new['score_z_diff_3-2'].abs()

    # get preds
    ld_preds_new = []
    for cmpname, models in d_models.items():
        for nrun, mdl in enumerate(models):
            new_prob1,new_prob2 = mdl.predict_proba(new_feats_z_df.values)[0]
            new_pred = mdl.predict(new_feats_z_df.values)[0]
            d_preds_new = {
                'comparison': cmpname,
                'run':nrun,
                'predict_type': 'new',
                'test_label': ' / '.join(mdl.classes_),
                'true_label': '',
                'pred_label': new_pred,
                'prob1': new_prob1,
                'prob2': new_prob2,
            }
            ld_preds_new.append(d_preds_new)
    df_preds_new = pd.DataFrame(ld_preds_new)
    
    return df_preds_new, df_feats_new

In [103]:
df_new_preds, df_new_feats = get_new_preds_feats(newtext)

,score_mean1,score_mean2,score_mean3,score_z1,score_z2,score_z3,score_mean_diff_3-1,score_z_diff_3-1,score_mean_diff_abs_3-1,score_z_diff_abs_3-1
feature,,,,,,,,,,
sent_num_words_in_independent_clauses,372.295918,526.388508,934.210526,-0.222854,0.571130,2.675274,561.914609,2.898128,561.914609,2.898128
sent_height,503.777726,460.239806,350.877193,0.181084,-0.522419,-2.312324,-152.900533,-2.493409,152.900533,2.493409
deprel_punct,119.032574,131.540991,188.596491,-0.115380,0.312503,2.293885,69.563918,2.409266,69.563918,2.409266
...,...,...,...,...,...,...,...,...,...,...
deprel_case,109.321220,122.004200,96.491228,-0.171463,0.502829,-0.868555,-12.829992,-0.697092,12.829992,0.697092
"pos_,",50.954631,61.219855,87.719298,-0.175766,0.487795,2.211173,36.764667,2.386939,36.764667,2.386939
deprel_nmod,52.988886,56.663200,74.561404,-0.047866,0.216127,1.511381,21.572518,1.559247,21.572518,1.559247


In [ ]:
l_feats_new = []
for feat in mdl.features_:
    d_feats_new = {
        
    }
    l_feats_new.append(d_feats_new)
    if len(l_feats_new) > 10:
        break
l_feats_new


,feature,weight,run,feat_desc,comparison,group1,group2,score_mean1,score_mean2,score_z1,...,score_mean_diff_pct,score_mean_div,score_mean_div_abs,score_z_diff,score_z_diff_abs,score_z_diff_pct,score_z_div,score_z_div_abs,feat_name,feat_type
51,pos_.,1.169448,4.5,,1925-1950 Philosophy vs 1925-1950 Literature,1925-1950 Philosophy,1925-1950 Literature,36.660937,39.852607,-0.008782,...,-0.080087,0.919913,0.919913,-0.336781,0.336781,-1.026773,-0.026773,0.026773,.,pos
29,deprel_mark,1.153033,4.5,,2000-2025 Philosophy vs 2000-2025 Literature,2000-2025 Philosophy,2000-2025 Literature,49.018079,32.569007,0.477035,...,0.505053,1.505053,1.505053,1.155853,1.155853,-1.702743,-0.702743,0.702743,mark,deprel
13,deprel_compound,1.040559,4.5,,1975-2000 Philosophy vs 1975-2000 Literature,1975-2000 Philosophy,1975-2000 Literature,15.640265,12.428111,0.014259,...,0.258459,1.258459,1.258459,0.306071,0.306071,-1.048865,-0.048865,0.048865,compound,deprel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49,pos_-LRB-,-1.020320,4.5,,1925-1950 Philosophy vs 1925-1950 Literature,1925-1950 Philosophy,1925-1950 Literature,1.955252,3.009010,-0.503397,...,-0.350201,0.649799,0.649799,-0.190530,0.190530,0.608983,1.608983,1.608983,-LRB-,pos
68,pos_NNP,-1.143318,4.5,,1950-1975 Philosophy vs 1950-1975 Literature,1950-1975 Philosophy,1950-1975 Literature,16.009598,46.943251,-0.472309,...,-0.658958,0.341042,0.341042,-1.304111,1.304111,-1.567814,-0.567814,0.567814,NNP,pos
68,pos_NNP,-1.634573,4.5,,1925-1950 Philosophy vs 1925-1950 Literature,1925-1950 Philosophy,1925-1950 Literature,13.682500,58.235551,-0.575356,...,-0.765049,0.234951,0.234951,-1.887837,1.887837,-1.438373,-0.438373,0.438373,NNP,pos
